In [8]:
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQ_GDXRfpLMoxW430-XRe5anOFIyE0NXYU44dLd57Y3GiQzSUciaxg5ep-IZcdVfJx8kqbXSIoM5U85/pub?output=csv' -O w_train.csv

--2023-01-12 13:06:03--  https://docs.google.com/spreadsheets/d/e/2PACX-1vQ_GDXRfpLMoxW430-XRe5anOFIyE0NXYU44dLd57Y3GiQzSUciaxg5ep-IZcdVfJx8kqbXSIoM5U85/pub?output=csv
Resolving docs.google.com (docs.google.com)... 142.251.163.100, 142.251.163.101, 142.251.163.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.163.100|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0k-94-sheets.googleusercontent.com/pub/8hg3b1ek8ops3o87ml9dh1hpus/2k0annf25d4kl1vpse7uchqrjg/1673528760000/116566562534349565554/*/e@2PACX-1vQ_GDXRfpLMoxW430-XRe5anOFIyE0NXYU44dLd57Y3GiQzSUciaxg5ep-IZcdVfJx8kqbXSIoM5U85?output=csv [following]
--2023-01-12 13:06:03--  https://doc-0k-94-sheets.googleusercontent.com/pub/8hg3b1ek8ops3o87ml9dh1hpus/2k0annf25d4kl1vpse7uchqrjg/1673528760000/116566562534349565554/*/e@2PACX-1vQ_GDXRfpLMoxW430-XRe5anOFIyE0NXYU44dLd57Y3GiQzSUciaxg5ep-IZcdVfJx8kqbXSIoM5U85?output=csv
Resolving doc-0k-94-sheets.googleusercontent.com (d

In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [11]:
import pandas as pd
train = pd.read_csv('w_train.csv')
train.head(10)

,tweet,polarity
0,needs to learn to share the desk and have a ...,positive
1,same that murdered 7 in after bomber killed 13,negative
2,i think its for to learn to analyze rather tha...,positive
3,russia lost 27 ka52 attack helicopters in the ...,negative
4,starved and ignored Ethiopian lecturers need i...,negative
5,are you really dumb enough to think that anyt...,negative
6,how genes drive your dogs lovable and wacky be...,positive
7,card games are boring so how did two of them b...,negative
8,liars crooks and frauds,negative
9,fuck fuck fuck,negative


In [12]:
df = pd.DataFrame(train,columns=['tweet','porality'])
df

,tweet,porality
0,needs to learn to share the desk and have a ...,NaN
1,same that murdered 7 in after bomber killed 13,NaN
2,i think its for to learn to analyze rather tha...,NaN
3,russia lost 27 ka52 attack helicopters in the ...,NaN
4,starved and ignored Ethiopian lecturers need i...,NaN
...,...,...
494,"In Finland , metal sector company Componenta i...",NaN
495,"Member of the HK Ruokatalo Group in Finland , ...",NaN
496,"The largest construction company in Finland , ...",NaN
497,"According to Aktia Asset Management , Finnish ...",NaN


In [13]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [14]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['sentiment'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

In [15]:
def AddFormat(text,sentiment,reviews_train):
  format = {'review': text,'sentiment': sentiment}
  reviews_train.append(format)
  return 0

In [16]:
reviews_train = []
data = np.array(df)
for i in data :
  AddFormat(i[0],i[1],reviews_train)

df = pd.DataFrame(reviews_train)

print(reviews_train)
# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

[{'review': 'needs to learn to share the desk  and  have a lot to add', 'sentiment': nan}, {'review': 'same that murdered 7 in after bomber killed 13', 'sentiment': nan}, {'review': 'i think its for to learn to analyze rather than blindly believe', 'sentiment': nan}, {'review': 'russia lost 27 ka52 attack helicopters in the ukraine invasion', 'sentiment': nan}, {'review': 'starved and ignored Ethiopian lecturers need international media coverage', 'sentiment': nan}, {'review': 'are you really dumb enough  to think that anything is permanent now i guess paul can get remarried', 'sentiment': nan}, {'review': 'how genes drive your dogs lovable and wacky behavior', 'sentiment': nan}, {'review': 'card games are boring so how did two of them become my favourites of 2022', 'sentiment': nan}, {'review': 'liars crooks and frauds', 'sentiment': nan}, {'review': 'fuck fuck fuck', 'sentiment': nan}, {'review': 'Genius UK Business Uses Christmas Trees to Protect the Region From Flooding', 'sentimen

In [17]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
16/16 [==============================] - 3s 27ms/step - loss: 0.4834 - accuracy: 0.9379
Epoch 2/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0597 - accuracy: 1.0000
Epoch 3/50
16/16 [==============================] - 0s 26ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 4/50
16/16 [==============================] - 0s 26ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 5/50
16/16 [==============================] - 0s 26ms/step - loss: 7.5838e-04 - accuracy: 1.0000
Epoch 6/50
16/16 [==============================] - 0s 26ms/step - loss: 5.9402e-04 - accuracy: 1.0000
Epoch 7/50
16/16 [==============================] - 0s 26ms/step - loss: 4.9343e-04 - accuracy: 1.0000
Epoch 8/50
16/16 [==============================] - 0s 26ms/step - loss: 4.2160e-04 - accuracy: 1.0000
Epoch 9/50
16/16 [==============================] - 0s 27ms/step - loss: 3.6685e-04 - accuracy: 1.0000
Epoch 10/50
16/16 [==============================] - 0s 26ms/step - loss: 3.2365

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                36224     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


In [22]:
cnn_reviews_test = [
         {'review': 'needs to learn to share the desk  and  have a lot to add', 'sentiment': 'positive'},
         {'review': 'everyone can be a hero.', 'sentiment': 'positive'},
         {'review': 'i think its for to learn to analyze rather than blindly believe', 'sentiment': 'positive'},
         {'review': 'russia lost 27 ka52 attack helicopters in the ukraine invasion', 'sentiment': 'negative'},
         {'review': 'card games are boring so how did two of them become my favourites of 2022.', 'sentiment': 'negative'},
         {'review': 'fuck', 'sentiment': 'negative'}
    ]
test_df = pd.DataFrame(cnn_reviews_test)

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 0s - loss: 5.1110 - accuracy: 0.5000 - 23ms/epoch - 23ms/step
Test score: 5.11102819442749
Test accuracy: 0.5
